This is a bare-bones file that demonstrates downloading a dataset from google drive and using it to train a YOLOv11 model.

Consider this a placeholder only - if you're training a model for real, please overhaul this file however you deem necessary!

In [ ]:
!pip install -U ultralytics gdown

In [ ]:
google_drive_file_id = '1N__ZI4KcmXmDADv4dYsKhIIZoe5-4iZg'  # <-- From sharing link in google drive, eg https://drive.google.com/file/d/1N__ZI4KcmXmDADv4dYsKhIIZoe5-4iZg/view?usp=sharing
google_drive_file_name = '20250623_merged_yolo_reshuffled.zip'

In [ ]:
import zipfile
from pathlib import Path
import gdown
from ultralytics import YOLO
import yaml

project_root = Path('/content')
dataset_root_name = 'yolo_dataset'
output_path = project_root / google_drive_file_name
extract_dir = project_root / dataset_root_name
dataset_root_path = Path('/content') / dataset_root_name
yaml_path = dataset_root_path / 'data.yaml'

# Download from Google Drive and extract
gdown.download(id=google_drive_file_id, output=str(output_path), quiet=False)
print(f"Dataset downloaded from google drive: {output_path}\n")

In [ ]:
# Extract files
print("Extracting...\n")
with zipfile.ZipFile(output_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)
print(f"Dataset extracted to: {extract_dir}\n", )

In [ ]:
# Update path in data.yaml to work with where we've unzipped it
print("Updating data.yaml path...\n")
with yaml_path.open('r') as file:
    data = yaml.safe_load(file)

data['path'] = str(dataset_root_path)
data['train'] = str(dataset_root_path / 'images' / 'train')
data['val'] = str(dataset_root_path / 'images' / 'val')
data['test'] = str(dataset_root_path / 'images' / 'test')

with yaml_path.open('w') as file:
    yaml.dump(data, file)

print(f"Updated path in data.yaml to: {data['path']}")

In [ ]:
# Convert_class0_to_80.py
import os

# === CONFIGURATION ===
LABELS_DIR = "/content/yolo_dataset/labels"  # Change this to your labels directory

def convert_labels(root_dir):
    updated_files = 0
    for dirpath, _, filenames in os.walk(root_dir):
        for file in filenames:
            if file.endswith(".txt"):
                file_path = os.path.join(dirpath, file)
                with open(file_path, "r") as f:
                    lines = f.readlines()

                new_lines = []
                modified = False
                for line in lines:
                    parts = line.strip().split()
                    if parts and parts[0] == "0":
                        parts[0] = "80"
                        modified = True
                    new_lines.append(" ".join(parts) + "\n")

                if modified:
                    with open(file_path, "w") as f:
                        f.writelines(new_lines)
                    updated_files += 1

    print(f"[✔] Updated {updated_files} files with class ID 80.")

if __name__ == "__main__":
    convert_labels(LABELS_DIR)

In [ ]:
# Sanity check that it's possible to train a model with the dataset
dataset_yaml = Path('/content/yolo_dataset/data.yaml')
model = YOLO('yolo11n.pt')
results = model.train(data=str(dataset_yaml), epochs=1, imgsz=640, batch=16, freeze=10)

In [ ]:
# Restore the original COCO classes that YOLO was trained on into th. https://github.com/ultralytics/ultralytics/blob/main/ultralytics/cfg/datasets/coco.yaml

# Update data.yaml to include the new class
with yaml_path.open('r') as file:
    data = yaml.safe_load(file)

# Original COCO class names from the prompt
original_coco_names = {
    0: 'person', 1: 'bicycle', 2: 'car', 3: 'motorcycle', 4: 'airplane',
    5: 'bus', 6: 'train', 7: 'truck', 8: 'boat', 9: 'traffic light',
    10: 'fire hydrant', 11: 'stop sign', 12: 'parking meter', 13: 'bench',
    14: 'bird', 15: 'cat', 16: 'dog', 17: 'horse', 18: 'sheep', 19: 'cow',
    20: 'elephant', 21: 'bear', 22: 'zebra', 23: 'giraffe', 24: 'backpack',
    25: 'umbrella', 26: 'handbag', 27: 'tie', 28: 'suitcase', 29: 'frisbee',
    30: 'skis', 31: 'snowboard', 32: 'sports ball', 33: 'kite', 34: 'baseball bat',
    35: 'baseball glove', 36: 'skateboard', 37: 'surfboard', 38: 'tennis racket',
    39: 'bottle', 40: 'wine glass', 41: 'cup', 42: 'fork', 43: 'knife',
    44: 'spoon', 45: 'bowl', 46: 'banana', 47: 'apple', 48: 'sandwich',
    49: 'orange', 50: 'broccoli', 51: 'carrot', 52: 'hot dog', 53: 'pizza',
    54: 'donut', 55: 'cake', 56: 'chair', 57: 'couch', 58: 'potted plant',
    59: 'bed', 60: 'dining table', 61: 'toilet', 62: 'tv', 63: 'laptop',
    64: 'mouse', 65: 'remote', 66: 'keyboard', 67: 'cell phone', 68: 'microwave',
    69: 'oven', 70: 'toaster', 71: 'sink', 72: 'refrigerator', 73: 'book',
    74: 'clock', 75: 'vase', 76: 'scissors', 77: 'teddy bear', 78: 'hair drier',
    79: 'toothbrush'
}


# Combine original names with the new class
all_class_names = original_coco_names
all_class_names[80] = 'trash'

# Update the 'names' field in the data dictionary
data['names'] = all_class_names

# Update the number of classes
data['nc'] = len(all_class_names) # Ensure nc matches the number of names

with yaml_path.open('w') as file:
    yaml.dump(data, file, sort_keys=False) # sort_keys=False preserves the order if possible

print(f"Updated data.yaml with all class names:")
!cat /content/yolo_dataset/data.yaml

In [ ]:
# Full COCO dataset download and merge
import os
import zipfile
import requests
from tqdm import tqdm
from pathlib import Path
import shutil

# === URLs for COCO 2017 ===
coco_image_zips = {
    "train": "http://images.cocodataset.org/zips/train2017.zip",
    "val": "http://images.cocodataset.org/zips/val2017.zip",
    "test": "http://images.cocodataset.org/zips/test2017.zip"
}

coco_labels_zip_url = "https://github.com/ultralytics/assets/releases/download/v0.0.0/coco2017labels.zip"

# === Target folders ===
base_dir = Path("/content/yolo_dataset")
image_dirs = {
    "train": base_dir / "images" / "train",
    "val": base_dir / "images" / "val",
    "test": base_dir / "images" / "test"
}
label_dirs = {
    "train": base_dir / "labels" / "train",
    "val": base_dir / "labels" / "val"
}

# === Temp folders ===
temp_zip_dir = Path("temp_zips")
temp_coco_dir = Path("temp_coco")
temp_zip_dir.mkdir(exist_ok=True)
temp_coco_dir.mkdir(exist_ok=True)

# === Download helper ===
def download_file(url, dest):
    if dest.exists():
        print(f"✅ {dest.name} already downloaded.")
        return
    print(f"⬇️ Downloading {url}")
    with requests.get(url, stream=True) as r:
        r.raise_for_status()
        with open(dest, 'wb') as f:
            for chunk in tqdm(r.iter_content(chunk_size=8192)):
                f.write(chunk)
    print(f"✅ Downloaded {dest.name}")

# === Download COCO image zips ===
for split, url in coco_image_zips.items():
    zip_path = temp_zip_dir / f"{split}2017.zip"
    download_file(url, zip_path)

# === Download COCO YOLO-format labels zip ===
labels_zip_path = temp_zip_dir / "coco2017labels.zip"
download_file(coco_labels_zip_url, labels_zip_path)

# === Extract image zips ===
for split in ["train", "val", "test"]:
    print(f"📦 Extracting COCO {split} images...")
    with zipfile.ZipFile(temp_zip_dir / f"{split}2017.zip", 'r') as zip_ref:
        zip_ref.extractall(temp_coco_dir)

# === Extract YOLO labels ===
print("📦 Extracting COCO YOLO-format labels...")
with zipfile.ZipFile(labels_zip_path, 'r') as zip_ref:
    zip_ref.extractall(temp_coco_dir)

# === Move images and labels to target folders ===
print("🚚 Moving images and labels to /content/yolo_dataset...")

for split in ["train", "val", "test"]:
    img_src = temp_coco_dir / f"{split}2017"
    img_dst = image_dirs[split]
    img_dst.mkdir(parents=True, exist_ok=True)
    # Add check for files in source directory
    print(f"Checking for image files in {img_src}: {len(list(img_src.glob('*.jpg')))} found.")
    for f in tqdm(list(img_src.glob("*.jpg")), desc=f"Copying {split} images"):
        shutil.move(str(f), img_dst / f.name)

for split in ["train", "val"]:
    # Corrected source path to include the 'coco/' subdirectory
    lbl_src = temp_coco_dir / "coco" / "labels" / f"{split}2017"
    lbl_dst = label_dirs[split]
    lbl_dst.mkdir(parents=True, exist_ok=True)
    # Add check for files in source directory
    print(f"Checking for label files in {lbl_src}: {len(list(lbl_src.glob('*.txt')))} found.")
    for f in tqdm(list(lbl_src.glob("*.txt")), desc=f"Copying {split} labels"):
        # Check if the source label file exists before attempting to move
        if f.exists():
            shutil.move(str(f), lbl_dst / f.name)
        else:
            print(f"⚠️ Warning: Label file not found for {f.name}. Skipping.")


# === Cleanup temp dirs ===
shutil.rmtree(temp_coco_dir)
print("\n✅ COCO dataset fully merged into /content/yolo_dataset/")

In [ ]:
# Train a YOLO11 model with the dataset
import shutil
from google.colab import files
from pathlib import Path
import torch
from ultralytics import YOLO
import yaml

dataset_yaml = Path('/content/yolo_dataset/data.yaml')

# Load a pre-trained YOLOv11n model
model = YOLO('yolo11n.pt') # If resuming training, change this path accordingly to point to the last.pt

# Get the number of original classes from the pre-trained model
original_classes = model.model.model[-1].nc
print(f"Original classes in the pre-trained model: {original_classes}")

# Modify the detection head to include the new class
# This requires accessing the last layer and adjusting its output size
# The last layer is typically the Detect layer

# Assuming the last layer is the Detect layer and has a 'nc' attribute
if hasattr(model.model.model[-1], 'nc'):
    # Get the current detection layer
    detect_layer = model.model.model[-1]

    # Get the configuration for the new detection layer
    new_nc = original_classes + 1

    # Directly modify the number of classes in the detect layer
    detect_layer.nc = new_nc

    # Also need to adjust the output shape for the classification head
    # This is typically done by modifying the shape of the classification head's output
    # within the Detect layer's forward pass or by recreating the classification head.
    # A more reliable way in YOLOv11 is to directly adjust the convolutional layers
    # that produce the classification output.

    # In YOLOv11 Detect layer, the classification output channels are typically
    # handled by a convolutional layer (often within a `cls` attribute)
    # with an output size of nc * na (number of classes * number of anchors).
    # Since 'na' is not directly accessible or used as before, we need to
    # adjust the output shape based on the new number of classes.

    # Let's try to modify the output channels of the classification convolutional layers
    # This is based on the assumption of a common YOLOv11 Detect layer structure.
    try:
        # Assuming detect_layer.cv2 is a ModuleList of classification convolutional layers
        # And assuming the output channels of these layers are proportional to the number of classes
        for i, cv in enumerate(detect_layer.cv2):
            # The output channels of the classification head convolution should be new_nc * na
            # Since 'na' is not directly available, let's assume the output shape is implicitly handled
            # by the number of classes and the input channels.

            # A simpler modification is to just set the `nc` attribute and rely on
            # the model's internal mechanisms or subsequent weight loading with strict=False.
            pass # We already set detect_layer.nc = new_nc
    except AttributeError:
        print("Could not find attribute 'cv2' in the Detect layer. Trying a different approach.")
        # If cv2 is not found, the structure is different.
        # We might need to look for other named attributes or the structure of submodules.

    # Let's revert to the original plan of setting nc and relying on strict=False loading
    # This requires having the pretrained state_dict available.
    # Load the pre-trained state dictionary again with strict=False
    # This might help transfer weights to the matching parts of the new layer
    # We need to get the state_dict of the original model first.
    original_model_for_state_dict = YOLO('yolo11n.pt')
    pretrained_state_dict = original_model_for_state_dict.state_dict()


    model.load_state_dict(pretrained_state_dict, strict=False)
    print("State dictionary loaded with strict=False after setting nc in the Detect layer.")


else:
    print("Could not find 'nc' attribute in the last layer. Cannot modify detection head.")


# Train the model
results = model.train(data=str(dataset_yaml), epochs=50, imgsz=640, batch=-1, freeze=10) # Change epochs as required. If resuming from a previously trained model, add resume=true. last.pt will be used.

print(f"Training complete. Results saved to: {model.trainer.save_dir}")

# Get the directory where results are saved
save_dir = Path(model.trainer.save_dir)
print(f"Training complete. Results saved to: {save_dir}")

# Zip the results directory
zip_path = save_dir.with_suffix('.zip')
shutil.make_archive(str(save_dir), 'zip', root_dir=save_dir)

# Download the zipped results
files.download(str(zip_path))